In [41]:
import pandas as pd
from datetime import timedelta
from datetime import datetime as dt
import datetime
import yfinance as yf
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import pytz

### Helper Functions

In [42]:
def next_four_fridays():
    # Get today's date and time in EST timezone
    est = pytz.timezone('US/Eastern')
    today = dt.now(est)
    
    # Find the next Friday
    days_until_friday = (4 - today.weekday() + 7) % 7
    next_friday = today + timedelta(days=days_until_friday)
    
    # Reset the time to 4:30 PM on the next Friday
    next_friday = next_friday.replace(hour=16, minute=30, second=0, microsecond=0)
    
    # If today is Friday and it's past 4:30 PM, consider the next week's Friday
    if today.weekday() == 4 and today.time() > next_friday.time():
        next_friday += timedelta(days=7)
    
    # Remove time from datetime for output
    next_friday = next_friday.date()
    
    # Find the Friday after the next
    following_friday = next_friday + timedelta(days=7)
    
    # Find the Friday after the following
    next_next_friday = following_friday + timedelta(days=7)
    
    # Find the Friday after the next next
    next_next_next_friday = next_next_friday + timedelta(days=7)
    
    return next_friday.strftime('%Y-%m-%d'), following_friday.strftime('%Y-%m-%d'), next_next_friday.strftime('%Y-%m-%d'), next_next_next_friday.strftime('%Y-%m-%d')

# Get the next four Fridays
friday1, friday2, friday3, friday4 = next_four_fridays()
print(f"The next four Fridays are: {friday1}, {friday2}, {friday3}, and {friday4}")

The next four Fridays are: 2023-09-08, 2023-09-15, 2023-09-22, and 2023-09-29


### Assuming Profits

In [53]:
# Initialize the variables
initial_amount = 1327  # Starting amount in dollars
roi_rate = 5

# Number of weeks for which the ROI is to be calculated
weeks = 38

# Initialize a variable to hold the current amount
current_amount = initial_amount

# Loop through each week to calculate and print the ROI
for week in range(1, weeks + 1):
    # Calculate the number of $100 chunks in the current amount
    chunks = current_amount // 100  # Use integer division to get whole chunks
    
    # Calculate the ROI for this week
    weekly_roi = chunks * (roi_rate)  # 1% of each $100 chunk is $1
    
    # Print the details for this week
    print(f"Week {week}: {current_amount} +{weekly_roi}")
    
    # Update the current amount for the next week
    current_amount += weekly_roi


Week 1: 1327 +65
Week 2: 1392 +65
Week 3: 1457 +70
Week 4: 1527 +75
Week 5: 1602 +80
Week 6: 1682 +80
Week 7: 1762 +85
Week 8: 1847 +90
Week 9: 1937 +95
Week 10: 2032 +100
Week 11: 2132 +105
Week 12: 2237 +110
Week 13: 2347 +115
Week 14: 2462 +120
Week 15: 2582 +125
Week 16: 2707 +135
Week 17: 2842 +140
Week 18: 2982 +145
Week 19: 3127 +155
Week 20: 3282 +160
Week 21: 3442 +170
Week 22: 3612 +180
Week 23: 3792 +185
Week 24: 3977 +195
Week 25: 4172 +205
Week 26: 4377 +215
Week 27: 4592 +225
Week 28: 4817 +240
Week 29: 5057 +250
Week 30: 5307 +265
Week 31: 5572 +275
Week 32: 5847 +290
Week 33: 6137 +305
Week 34: 6442 +320
Week 35: 6762 +335
Week 36: 7097 +350
Week 37: 7447 +370
Week 38: 7817 +390


# Viewing option chains

In [44]:
print(friday1)

2023-09-08


In [51]:
file_viewer = pd.read_csv('naked_puts_results.csv')
filtered_data = file_viewer[file_viewer['Expiration Date'] == friday1]

# Display all rows, no limit
pd.set_option('display.max_rows', None)
# Display all columns, no limit
pd.set_option('display.max_columns', None)

filtered_data['ROI (%)'] = filtered_data['ROI (%)'].astype(float)
# filter out rows with ROI (%) greater than 0.8
# filtered_data = filtered_data[filtered_data['ROI (%)'] > 0.8]
# filter out rows with strike price less than 6
# filtered_data = filtered_data[filtered_data['strike'] <= 7]

# sort by ROI (%) in ascending order
filtered_data = filtered_data.sort_values(by=['ROI (%)'], ascending=False)

# Show only the columns 'Expiration Date', 'ETF', 'Stock Price', 'bid', 'strike', 'ROI (%)', 'OTM (%)', 'Implied Volatility', 'volume', 'openInterest', 'highPrice', 'Original Stock Price', 'Original ROI (%)', 'Original OTM (%)', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', '52WeekHigh', and '52WeekLow'
filtered_data = filtered_data[['Expiration Date', 'ETF', 'Stock Price', 'bid', 'strike', 'ROI (%)', 'OTM (%)', 'Implied Volatility', 'volume', 'openInterest', 'highPrice', 'Original Stock Price', 'Original ROI (%)',
    'Original OTM (%)', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', '52WeekHigh', '52WeekLow']]

# Show only the top 10 rows
# filtered_data.head(10)

,Expiration Date,ETF,Stock Price,bid,strike,ROI (%),OTM (%),Implied Volatility,volume,openInterest,highPrice,Original Stock Price,Original ROI (%),Original OTM (%),50-day MA,100-day MA,200-day MA,RSI,MACD,VWAP,52WeekHigh,52WeekLow
18,2023-09-08,CGC,0.55,0.03,0.5,6.38,9.09,1.81,4595.0,3721.0,0.03,0.55,6.38,9.09,0.45,0.75,1.66,69.00,0.01,1.60,4.32,0.37
27,2023-09-08,EOSE,3.85,0.20,3.5,6.06,9.09,1.73,1243.0,1931.0,0.20,3.85,6.06,9.09,3.35,2.87,2.23,73.33,0.16,2.64,5.03,0.97
8,2023-09-08,AMC,13.10,0.74,13.0,6.04,0.76,1.04,26394.0,16135.0,0.74,13.10,6.04,0.76,39.30,44.89,50.28,23.70,-9.29,45.70,102.20,10.91
42,2023-09-08,HE,15.05,0.75,15.0,5.26,0.33,0.90,915.0,216.0,0.75,15.05,5.26,0.33,29.49,33.02,35.94,36.70,-5.32,18.77,41.60,9.66
23,2023-09-08,CHPT,7.44,0.32,7.0,4.79,5.91,1.21,519.0,2396.0,0.32,7.44,4.79,5.91,7.99,8.37,9.59,41.55,-0.26,10.67,19.25,6.79
57,2023-09-08,MVIS,2.51,0.10,2.5,4.17,0.40,0.75,599.0,312.0,0.10,2.51,4.17,0.40,3.47,3.51,3.06,30.11,-0.29,3.93,7.65,1.82
34,2023-09-08,GOEV,0.59,0.02,0.5,4.17,15.25,1.88,2613.0,1539.0,0.02,0.59,4.17,15.25,0.56,0.60,0.80,59.21,0.01,0.80,2.89,0.43
2,2023-09-08,AAOI,13.94,0.45,13.5,3.45,3.16,0.89,62.0,1.0,0.45,13.94,3.45,3.16,9.95,6.17,4.25,45.95,1.30,7.54,15.48,1.64
65,2023-09-08,NVAX,8.21,0.25,8.0,3.23,2.56,0.80,314.0,758.0,0.25,8.21,3.23,2.56,7.97,7.95,9.50,60.34,0.10,11.50,33.00,5.76
58,2023-09-08,NKLA,1.18,0.03,1.0,3.09,15.25,1.56,1010.0,33575.0,0.03,1.18,3.09,15.25,1.85,1.35,1.78,26.30,-0.24,1.92,5.42,0.54


In [46]:
# Get the dataFrame from 'Data/putsDataSuccessFailed.csv'
df = pd.read_csv('Data/putsDataSuccessFailed.csv')

# Define your color function for entire row
def color_row(row):
    color = 'lightgreen' if row['Status'] == 'Success' else 'lightcoral'
    return ['background-color: %s' % color]*len(row)

# Sort the rows by Original ROI(%) in ascending order
df = df.sort_values(by=['Original ROI (%)'], ascending=False)

# Get only the top 20 rows
top_20_df = df.head(20)

# Select only the specified columns
columns_to_show = ['ETF', 'Original Stock Price', 'Stock Price', 'strike', 'volume', 'openInterest', 'highPrice', 'Original ROI (%)', 'Original OTM (%)', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', '52WeekHigh', '52WeekLow', 'Status']

top_20_df_filtered = top_20_df[columns_to_show]

# Apply the color function to the entire DataFrame
styled_top_20_df = top_20_df_filtered.style.apply(color_row, axis=1).format("{:.2f}", subset=top_20_df_filtered.select_dtypes(include=['float64']).columns)

# Display the styled DataFrame
# styled_top_20_df

,ETF,Original Stock Price,Stock Price,strike,volume,openInterest,highPrice,Original ROI (%),Original OTM (%),50-day MA,100-day MA,200-day MA,RSI,MACD,VWAP,52WeekHigh,52WeekLow,Status
68,MULN,0.56,0.46,0.50,2430.00,5107.00,0.06,13.64,10.71,0.30,0.88,3.60,61.22,0.11,0.74,16.75,0.10,Failed
29,EOSE,3.69,3.85,3.50,748.00,489.00,0.35,11.11,5.15,3.35,2.84,2.19,79.25,0.09,2.55,5.03,0.97,Success
47,HE,13.88,15.05,13.00,863.00,5.00,0.80,7.88,6.34,32.00,34.92,37.70,18.37,-6.84,20.51,43.45,9.66,Success
46,HE,13.88,15.05,12.50,433.00,8.00,0.70,5.93,9.94,32.00,34.92,37.70,18.37,-6.84,20.51,43.45,9.66,Success
28,EOSE,3.69,3.85,3.00,874.00,2091.00,0.15,5.26,18.70,3.35,2.84,2.19,79.25,0.09,2.55,5.03,0.97,Success
122,UCAR,4.55,4.36,4.50,19.00,9.00,0.25,4.65,1.10,6.07,nan,nan,43.41,-0.45,8.42,43.18,3.11,Failed
8,AMC,11.87,13.10,11.00,2894.00,1001.00,0.48,4.36,7.33,41.93,46.36,51.33,17.60,-8.28,52.57,102.20,11.87,Success
35,GOEV,0.60,0.59,0.50,572.00,4630.00,0.01,4.17,16.67,0.55,0.60,0.82,57.09,-0.01,0.82,3.32,0.43,Success
16,BB,5.12,5.60,5.00,528.00,801.00,0.16,3.73,2.34,4.81,4.81,4.47,63.36,-0.00,4.54,6.20,3.18,Success
44,HE,13.90,15.05,11.50,130.00,118.00,0.45,3.60,17.27,32.00,34.92,37.70,18.39,-6.83,20.50,43.45,9.66,Success


In [47]:
# Columns for Machine Learning
columns = ['strike', 'volume', 'openInterest', 'highPrice', 'Original Stock Price', 'Original Implied Volatility', 'Original ROI (%)', 'Original OTM (%)', '50-day MA', '100-day MA', '200-day MA', 'RSI', 'MACD', 'VWAP', '52WeekHigh', '52WeekLow']

In [49]:
# Load your dataset
data = pd.read_csv("Data/putsDataSuccessFailed.csv")

# Preprocessing Steps
# -------------------
# 1. Fill missing values with median
# Fill missing values with median only for numeric columns
for col in data.select_dtypes(include=['float64']).columns:
    data[col].fillna(data[col].median(), inplace=True)

# 2. Label Encode 'Status' column
label_encoder = LabelEncoder()
data['Status'] = label_encoder.fit_transform(data['Status'])

# 3. Feature selection
selected_columns = ['strike', 'volume', 'openInterest', 'Implied Volatility', 'Original Stock Price', 'Original ROI (%)', 'Original OTM (%)', 'Original Implied Volatility', '50-day MA', '100-day MA', '200-day MA', 'RSI',
    'MACD', 'VWAP', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', '52WeekHigh', '52WeekLow', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'recommendationMean', 'recommendationKey',
    'numberOfAnalystOpinions']
X = data[selected_columns]
y = data['Status']

# 4. Feature Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Initialize the model
random_forest_model = RandomForestClassifier(random_state=42)

# Hyperparameter Tuning
rf_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid_search = GridSearchCV(estimator=random_forest_model, param_grid=rf_param_grid,
                              cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

rf_grid_search.fit(X_train_balanced, y_train_balanced)

# Get the best parameters and update the model
best_params = rf_grid_search.best_params_
random_forest_model = RandomForestClassifier(**best_params, random_state=42)

# Train the model with best parameters
random_forest_model.fit(X_train_balanced, y_train_balanced)

# Evaluate the model
y_pred = random_forest_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy with Best Parameters: {accuracy * 100:.2f}%")

# Making Predictions on New Data
# ------------------------------
# Load new data
new_data = pd.read_csv("naked_puts_results.csv")

# Apply the same preprocessing steps
for col in new_data.select_dtypes(include=['float64']).columns:
    new_data[col].fillna(new_data[col].median(), inplace=True)

new_X = new_data[selected_columns]
new_X_scaled = scaler.transform(new_X)

# Make predictions
new_predictions = random_forest_model.predict(new_X_scaled)

# Convert numerical predictions back to labels ("Success" or "Failed")
new_predictions_labels = label_encoder.inverse_transform(new_predictions)

# Add predictions to the new data
new_data["Guess"] = new_predictions_labels

# Save the new data with predictions
new_data.to_csv("Data/new_data_with_predictions.csv", index=False)

KeyError: "['targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'numberOfAnalystOpinions'] not in index"

In [ ]:
# Get the dataFrame from 'Data/new_data_with_predictions.csv'
df = pd.read_csv('Data/new_data_with_predictions.csv')

# Define your color function for entire row
def color_row(row):
    color = 'lightgreen' if row['Guess'] == 'Success' else 'lightcoral'
    return ['background-color: %s' % color]*len(row)

# Sort the rows by Original ROI(%) in ascending order
df = df.sort_values(by=['Original ROI (%)'], ascending=False)

# Get only the top 20 rows
top_20_df = df.head(40)

# Select only the specified columns
columns_to_show = ['Expiration Date', 'ETF', 'Stock Price', 'bid', 'strike', 'ROI (%)', 'OTM (%)', 'volume', 'openInterest', 'Guess']

top_20_df_filtered = top_20_df[columns_to_show]

# Apply the color function to the entire DataFrame
styled_top_20_df = top_20_df_filtered.style.apply(color_row, axis=1).format("{:.2f}", subset=top_20_df_filtered.select_dtypes(include=['float64']).columns)

# Display the styled DataFrame
styled_top_20_df

,Expiration Date,ETF,Stock Price,bid,strike,ROI (%),OTM (%),Implied Volatility,volume,openInterest,highPrice,Original Stock Price,Original ROI (%),Original OTM (%),50-day MA,100-day MA,200-day MA,RSI,MACD,VWAP,52WeekHigh,52WeekLow,Guess
19,2023-09-08,CGC,0.55,0.03,0.50,6.38,9.09,1.81,4595.00,3721.00,0.03,0.55,6.38,9.09,0.45,0.75,1.66,69.00,0.01,1.60,4.32,0.37,Success
28,2023-09-08,EOSE,3.85,0.20,3.50,6.06,9.09,1.73,1243.00,1931.00,0.20,3.85,6.06,9.09,3.35,2.87,2.23,73.33,0.16,2.64,5.03,0.97,Success
8,2023-09-08,AMC,13.10,0.74,13.00,6.04,0.76,1.04,26394.00,16135.00,0.74,13.10,6.04,0.76,39.30,44.89,50.28,23.70,-9.29,45.70,102.20,10.91,Success
43,2023-09-08,HE,15.05,0.75,15.00,5.26,0.33,0.90,915.00,216.00,0.75,15.05,5.26,0.33,30.15,33.92,37.20,35.84,-5.55,19.16,43.45,9.66,Success
24,2023-09-08,CHPT,7.44,0.32,7.00,4.79,5.91,1.21,519.00,2396.00,0.32,7.44,4.79,5.91,7.99,8.37,9.59,41.55,-0.26,10.67,19.25,6.79,Failed
35,2023-09-08,GOEV,0.59,0.02,0.50,4.17,15.25,1.88,2613.00,1539.00,0.02,0.59,4.17,15.25,0.56,0.60,0.80,59.21,0.01,0.80,2.89,0.43,Success
59,2023-09-08,MVIS,2.51,0.10,2.50,4.17,0.40,0.75,599.00,312.00,0.10,2.51,4.17,0.40,3.47,3.51,3.06,30.11,-0.29,3.93,7.65,1.82,Failed
2,2023-09-08,AAOI,13.94,0.45,13.50,3.45,3.16,0.89,62.00,1.00,0.45,13.94,3.45,3.16,9.95,6.17,4.25,45.95,1.30,7.54,15.48,1.64,Success
72,2023-09-08,NVAX,8.21,0.25,8.00,3.23,2.56,0.80,314.00,758.00,0.25,8.21,3.23,2.56,7.97,7.95,9.50,60.34,0.10,11.50,33.00,5.76,Failed
65,2023-09-08,NKLA,1.18,0.03,1.00,3.09,15.25,1.56,1010.00,33575.00,0.03,1.18,3.09,15.25,1.85,1.35,1.78,26.30,-0.24,1.92,5.42,0.54,Success
